# 🌙 Corrosion Insight  
### Modelo Predictivo de Riesgo de Corrosión a partir del Clima

Este notebook implementa un prototipo de **modelo de riesgo de corrosión ambiental** usando datos reales de clima obtenidos desde la API de **OpenWeather** (y, opcionalmente, su API de calidad de aire).

La idea es simular el tipo de pipeline que se podría usar en un contexto de **integridad de activos / mantenimiento predictivo** en energía u oil & gas:

---

## 🔗 Qué hace este notebook

- ☁️ **Ingesta de datos**  
  - Pronóstico meteorológico cada 3 horas (temperatura, humedad, viento, nubes, presión).  
  

- 🧪 **Construcción del dataset**  
  - Dataset temporal con variables ambientales.  
  - Features de calendario: hora del día, día de la semana, etc.

- ⚙️ **Simulación de la métrica de corrosión**  
  - Se genera una variable `corrosion_rate` que aumenta con:
    - mayor humedad,
    - mayor velocidad del viento,
    - mayor cobertura de nubes,  
    y se penaliza con temperaturas extremas.  

- 🧠 **Modelado y evaluación**  
  - Entrenamiento de modelos de regresión supervisada:
    - `LinearRegression`
    - `RandomForestRegressor`
  - Evaluación con **MAE (Mean Absolute Error)** para comparar desempeño.

- 📊 **Visualización**  
  - Gráfico interactivo en modo oscuro (**Plotly**) comparando:
    - **corrosión real (simulada)** vs **corrosión predicha** a lo largo del tiempo.

---

> 💡 **Nota importante**  
> En un entorno real, la columna `corrosion_rate` se reemplazaría por datos industriales:
> - inspecciones,
> - estados de activos,
> - fallas,
> - potenciales de protección, etc.  
> El pipeline sería el mismo y permitiría:
> - priorizar inspecciones,  
> - disparar alertas ante condiciones críticas,  
> - apoyar decisiones de **mantenimiento predictivo** en energía, oil & gas e infraestructura.

In [23]:
# Importación de librerías
import os                     #Liberia para traer funciones del os
from pathlib import Path      #Libreria encargada de la gestion de archivos
import numpy as np            #Libreria numpy para gestion de calculos matematicos
import pandas as pd           #Libreria pandas para gestion de DataFrames
import requests               #Libreria encargada de realizar las llamada al API

## 🔧 Configuración inicial: ubicación, API key y rutas de trabajo

En esta sección se define:

- La **ubicación geográfica** que se va a usar para consultar la API de OpenWeather (Cipolletti).
- La **API key** personal de OpenWeather, necesaria para autenticar las solicitudes.
- Las **rutas de trabajo** del proyecto:
  - carpeta raíz del proyecto,
  - subcarpeta `data/` para almacenar archivos,
  - archivo `forecast.csv` donde se guarda un snapshot del pronóstico.

Esta configuración deja listo el entorno para:
- reutilizar el mismo dataset sin llamar siempre a la API, y  
- mantener todos los datos organizados dentro de la carpeta `data/`.

In [ ]:
# 📍 Ubicación consumida por el API
city_name = "Cipolletti, AR"
lat = -38.94
lon = -67.99

# 🔑 API key de OpenWeather
OPENWEATHER_API_KEY = "your_openweather_api_key_here"


# Chequeo defensivo: si la API key está vacía, frenamos la ejecución
if not OPENWEATHER_API_KEY:
    raise ValueError("OpenWeather API key not found")


# 📂 Configuración de rutas para guardar datos en disco

BASE_DIR = Path().resolve()          # Carpeta base del proyecto (directorio actual, en ruta absoluta)
DATA_DIR = BASE_DIR / "data"         # Subcarpeta 'data' dentro del proyecto
DATA_DIR.mkdir(exist_ok=True)         # Crea la carpeta 'data' si no existe (no falla si ya está creada)

FORECAST_CSV = DATA_DIR / "forecast.csv"    # Ruta completa al archivo donde se guardará el pronóstico

BASE_DIR, DATA_DIR, FORECAST_CSV            # Mostrar las rutas para ver que quedaron bien


(PosixPath('/content'),
 PosixPath('/content/data'),
 PosixPath('/content/data/forecast.csv'))

## 🌤️ Función de ingesta: consulta a OpenWeather y arma el DataFrame de clima

En esta sección se define la función `fetch_openweather_forecast`, responsable de:

- Conectarse a la API **5-day / 3-hour forecast** de **OpenWeather** usando:
  - la **API key** configurada previamente,
  - la **latitud** y **longitud** de la ubicación de interés.
- Tomar la respuesta en formato **JSON** y transformar solo la información relevante en una estructura tabular:
  - ciudad y país,
  - fecha y hora del pronóstico (`timestamp`),
  - temperatura (`temp`),
  - humedad (`humidity`),
  - velocidad y dirección del viento (`wind_speed`, `wind_deg`),
  - porcentaje de nubosidad (`clouds`).
- Devolver un **`pandas.DataFrame` ordenado temporalmente**, listo para ser utilizado en los siguientes pasos:
  - construcción del dataset de corrosión,
  - generación de features adicionales,
  - entrenamiento de los modelos de regresión.

Esta función encapsula toda la lógica de ingesta desde la API, de modo que el resto del notebook pueda trabajar directamente con datos limpios sin preocuparse por el formato del JSON original.

In [26]:
#  Llama a la API 5-day/3-hour forecast de OpenWeather y devuelve un DataFrame con variables climáticas clave.

def fetch_openweather_forecast(api_key: str, lat: float, lon: float) -> pd.DataFrame:

  if not api_key:
    raise ValueError("OpenWeather API no esta configurada")      # Si la API key está vacía, corto la ejecución con un error claro


  url = f"https://api.openweathermap.org/data/2.5/forecast"      # Parámetros de la query: ubicación, API key, unidades y lenguaje
  params = {
    "lat": lat,
    "lon": lon,
    "appid": api_key,
    "units": "metric",
    "lang":"es"}

  resp = requests.get(url, params=params)                        # Realiza la solicitud HTTP GET a la API con los parámetros definidos
  resp.raise_for_status()                                        # Si la respuesta no es 200, lanza un error (defensivo)
  data = resp.json()                                             # Convierte la respuesta JSON

  records = []                                                   # Crea una lista vacía para almacenar los registros del DataFrame

  for item in data.get("list",[]):                               # Recorre cada elemento de la lista de pronósticos (cada 3 horas)
    main = item.get("main", {})
    wind = item.get("wind", {})
    clouds = item.get("clouds", {})
    weather = item.get("weather", [{}])[0]

    records.append({
        "city": data.get("city", {}).get("name"),                 # nombre de la ciudad (viene fuera de 'list')
        "country": data.get("city", {}).get("country"),           # país
        "timestamp": pd.to_datetime(item.get("dt_txt")),          # fecha y hora del pronóstico
        "temp": main.get("temp"),                                 # temperatura en °C
        "humidity": main.get("humidity"),                         # humedad relativa (%)
        "wind_speed": wind.get("speed"),                          # velocidad del viento
        "wind_deg": wind.get("deg"),                              # dirección del viento (grados)
        "clouds": clouds.get("all"),                              # porcentaje de nubosidad
    })
  return pd.DataFrame(records).sort_values("timestamp").reset_index(drop=True)         # Retorno los valores en un DataFrame ordenados por tiempo

df = fetch_openweather_forecast(OPENWEATHER_API_KEY, lat, lon)                         # Llamada a la función usando la API key y la ubicación definida antes

display(df)


,city,country,timestamp,temp,humidity,wind_speed,wind_deg,clouds
0,Cipolletti,AR,2025-11-11 06:00:00,17.76,64,6.81,212,23
1,Cipolletti,AR,2025-11-11 09:00:00,16.24,51,12.25,230,12
2,Cipolletti,AR,2025-11-11 12:00:00,17.40,33,13.06,242,5
3,Cipolletti,AR,2025-11-11 15:00:00,19.73,33,13.93,244,0
4,Cipolletti,AR,2025-11-11 18:00:00,22.17,25,16.47,248,16
5,Cipolletti,AR,2025-11-11 21:00:00,20.58,28,14.98,247,12
6,Cipolletti,AR,2025-11-12 00:00:00,16.71,42,12.69,248,7
7,Cipolletti,AR,2025-11-12 03:00:00,15.04,47,11.56,251,0
8,Cipolletti,AR,2025-11-12 06:00:00,13.90,50,10.02,244,0
9,Cipolletti,AR,2025-11-12 09:00:00,12.96,52,8.19,239,0


## 💾 Función de caché: usar el CSV local o llamar a la API

La función `load_or_fetch_forecast` se encarga de gestionar **de dónde** se obtienen los datos de clima:

- Si ya existe un archivo CSV con el pronóstico (`forecast.csv`) y **no** se fuerza la actualización:
  - carga directamente los datos desde el CSV,
  - evita hacer una nueva llamada a la API,
  - hace que el notebook sea **reproducible** y funcione incluso **sin conexión**.
- Si el CSV no existe, o se indica `force_refresh=True`:
  - llama a la API de OpenWeather a través de `fetch_openweather_forecast`,
  - guarda un **snapshot** del pronóstico en disco,
  - devuelve el DataFrame actualizado.

De esta forma, el resto del notebook puede trabajar siempre con un `DataFrame` llamado `df` (o `forecast_raw`) sin preocuparse de si los datos vinieron de disco o de la API.

In [54]:


# Carga el pronóstico desde un CSV local si está disponible, o, en caso contrario (o si se fuerza refresh), llama a la API y guarda un snapshot en disco.

def load_or_fetch_forecast(
    api_key: str,
    lat: float,
    lon: float,
    csv_path: Path = FORECAST_CSV,
    force_refresh: bool = False,
) -> pd.DataFrame:

    if csv_path.exists() and not force_refresh:                         # Caso 1: el archivo existe y NO queremos refrescar datos desde la API
        print(f"Cargando pronóstico desde {csv_path}")
        df = pd.read_csv(csv_path)
    else:
        print(f"Llamando a OpenWeather API y guardando snapshot...")    # Caso 2: el archivo no existe o queremos forzar actualización
        df = fetch_openweather_forecast(api_key, lat, lon)              # Obtenemos el DataFrame directamente desde la API
        df.to_csv(csv_path, index=False)                                # Guardamos un snapshot en disco para poder reutilizarlo luego
        print(f"Pronóstico guardado en {csv_path}")
    return df                                                           # Devolvemos siempre un DataFrame listo para usar

df = load_or_fetch_forecast(OPENWEATHER_API_KEY, lat, lon)              # Uso de la función: obtengo el pronóstico ya sea desde CSV o desde la API
display(df)

Cargando pronóstico desde /content/data/forecast.csv


,city,country,timestamp,temp,humidity,wind_speed,wind_deg,clouds
0,Cipolletti,AR,2025-11-11 03:00:00,20.03,56,3.35,25,20
1,Cipolletti,AR,2025-11-11 06:00:00,19.09,54,6.81,212,23
2,Cipolletti,AR,2025-11-11 09:00:00,16.91,46,12.25,230,12
3,Cipolletti,AR,2025-11-11 12:00:00,17.40,33,13.06,242,5
4,Cipolletti,AR,2025-11-11 15:00:00,19.73,33,13.93,244,0
5,Cipolletti,AR,2025-11-11 18:00:00,22.17,25,16.47,248,16
6,Cipolletti,AR,2025-11-11 21:00:00,20.58,28,14.98,247,12
7,Cipolletti,AR,2025-11-12 00:00:00,16.71,42,12.69,248,7
8,Cipolletti,AR,2025-11-12 03:00:00,15.04,47,11.56,251,0
9,Cipolletti,AR,2025-11-12 06:00:00,13.90,50,10.02,244,0


## 📊 Carga final del pronóstico y vista rápida del DataFrame

En este bloque se ejecuta el flujo completo de ingesta de datos:

- Se llama a `load_or_fetch_forecast` para obtener el pronóstico:
  - desde el archivo `forecast.csv` si ya existe y no se fuerza actualización, o
  - directamente desde la API de OpenWeather si es la primera vez o si se pide `force_refresh=True`.
- El resultado se guarda en el DataFrame `forecast_raw`, que contiene:
  - una fila por cada intervalo de 3 horas,
  - variables climáticas listas para ser transformadas en un dataset de corrosión.
- Se imprime:
  - el **tamaño** del DataFrame (`shape`) para ver cuántas filas y columnas hay,
  - las primeras filas (`head()`) para inspeccionar visualmente que los datos se vean correctos.

A partir de `forecast_raw`, en los siguientes pasos se construirá el dataset enriquecido con features de calendario y la tasa de corrosión simulada.

In [30]:
# Ejecuta la función de caché/ingesta:

forecast_raw = load_or_fetch_forecast(
    api_key=OPENWEATHER_API_KEY,
    lat=lat,
    lon=lon,
    csv_path=FORECAST_CSV,
    force_refresh=False,)
print(forecast_raw.shape)
forecast_raw.head()

Cargando pronóstico desde /content/data/forecast.csv
(40, 8)


,city,country,timestamp,temp,humidity,wind_speed,wind_deg,clouds
0,Cipolletti,AR,2025-11-11 03:00:00,20.03,56,3.35,25,20
1,Cipolletti,AR,2025-11-11 06:00:00,19.09,54,6.81,212,23
2,Cipolletti,AR,2025-11-11 09:00:00,16.91,46,12.25,230,12
3,Cipolletti,AR,2025-11-11 12:00:00,17.40,33,13.06,242,5
4,Cipolletti,AR,2025-11-11 15:00:00,19.73,33,13.93,244,0


## 🧪 Construcción del dataset de corrosión (`corrosion_df`)

A partir del DataFrame `forecast_raw` con el pronóstico climático cada 3 horas, en esta sección se construye el dataset enriquecido que va a alimentar al modelo de regresión.

Lo que se hace puntualmente:

- ✅ **Copiar y normalizar el DataFrame base**
  - Se asegura que `timestamp` esté en formato `datetime`.
  - Se agrega la columna `city` para identificar la serie.

- 🕒 **Features de calendario + tiempo cíclico**
  - `hour`: hora del día (0–23).
  - `dayofweek`: día de la semana (0 = lunes).
  - `hour_sin` y `hour_cos`: codificación cíclica de la hora, para que el modelo entienda que 23:00 y 0:00 están “cerca” en el ciclo diario.

- 🏭 **Índice sintético de contaminación (`pollution_index`)**
  - Se asume una zona industrial en una determinada dirección (por ejemplo, oeste = 270°).
  - Se calcula qué tanto se alinea la dirección del viento con esa zona.
  - Se combina esa alineación con la velocidad del viento.
  - El resultado es un `pollution_index` mayor cuando el viento viene “desde la zona industrial” y trae más contaminantes hacia el activo.

- ⚙️ **Simulación de la tasa de corrosión (`corrosion_rate`)**
  - Se construye una base que aumenta con:
    - **humedad** (`humidity`),
    - **velocidad del viento** (`wind_speed`),
    - **nubosidad** (`clouds`),
    - **índice de contaminación** (`pollution_index`),
    y se penaliza cuando la temperatura se aleja de ~20 °C.
  - Se agrega ruido aleatorio controlado para que la relación no sea perfectamente determinista.
  - Se recorta a valores positivos y se redondea a 2 decimales.

El objetivo no es modelar corrosión real de forma exacta, sino generar una variable objetivo **coherente físicamente** con el clima y la presencia de contaminantes, suficiente para demostrar un pipeline completo de regresión aplicada a integridad de activos.

In [66]:
def build_corrosion_dataset(forecast_df: pd.DataFrame) -> pd.DataFrame:

    df = forecast_df.copy()

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["city"] = df["city"].astype(str)

    df["hour"] = df["timestamp"].dt.hour                            # Features de calendario
    df["dayofweek"] = df["timestamp"].dt.dayofweek

    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)            # Encoding cíclico de la hora (0–23)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)


    # --- Índice sintético de contaminación atmosférica ---

    industrial_dir = 270                                                                # Suposición: hay una zona industrial al oeste (270°).

    angle_diff = np.abs(((df["wind_deg"] - industrial_dir + 180 )% 360 ) - 180)         # # Distancia angular mínima entre la dirección del viento y 270°


    dir_factor = 1 - (angle_diff / 180.0)                                               # Mapeamos a [0,1]: 1 = viento desde la zona industrial (270°), 0 = dirección opuesta
    dir_factor = dir_factor.clip(lower=0)                                               # Recortamos a [0,1] dejando solo valores positivos o 0

    df["pollution_index"] = (dir_factor * df["wind_speed"]).round(3)                    # Determinamos que tanto viento contaminado viene de la zona industrial


     # --- Simulación de la tasa de corrosión ---

    base = (
        0.03 * df["humidity"]              # humedad alta favorece corrosión
        + 0.05 * df["wind_speed"]          # viento como transporte de contaminantes
        + 0.02 * df["clouds"]              # nubosidad ligada a condensación
        - 0.02 * (df["temp"] - 20).abs()   # penaliza alejarse de temp moderadas
        + 0.4  * df["pollution_index"]     # aporte extra por contaminación sintética
    )


    # Ruido aleatorio para evitar una relación determinista perfecta
    rng = np.random.default_rng(seed=42)
    noise = rng.normal(loc=0.0, scale=0.4, size=len(df))

    df["corrosion_rate"] = base + noise                                   # Sumamos un ruido aleatorio (media 0) para que los datos no sean perfectamente deterministas
    df["corrosion_rate"] = df["corrosion_rate"].clip(lower=0).round(3)    # Dejamos los valores positivos o en 0




    cols = [
        "timestamp",
        "city",
        "temp",
        "humidity",
        "wind_speed",
        "wind_deg",
        "clouds",
        "hour",
        "dayofweek",
        "pollution_index",
        "corrosion_rate",
        "hour_sin",
        "hour_cos",
        ]

    return df[cols].sort_values("timestamp").reset_index(drop=True)



build_corrosion_dataset(forecast_raw)

,timestamp,city,temp,humidity,wind_speed,wind_deg,clouds,hour,dayofweek,pollution_index,corrosion_rate,hour_sin,hour_cos
0,2025-11-11 03:00:00,Cipolletti,20.03,56,3.35,25,20,3,1,1.210,2.853,7.071068e-01,7.071068e-01
1,2025-11-11 06:00:00,Cipolletti,19.09,54,6.81,212,23,6,1,4.616,3.833,1.000000e+00,6.123234e-17
2,2025-11-11 09:00:00,Cipolletti,16.91,46,12.25,230,12,9,1,9.528,6.282,7.071068e-01,-7.071068e-01
3,2025-11-11 12:00:00,Cipolletti,17.40,33,13.06,242,5,12,1,11.028,6.478,1.224647e-16,-1.000000e+00
4,2025-11-11 15:00:00,Cipolletti,19.73,33,13.93,244,0,15,1,11.918,5.668,-7.071068e-01,-7.071068e-01
5,2025-11-11 18:00:00,Cipolletti,22.17,25,16.47,248,16,18,1,14.457,7.112,-1.000000e+00,-1.836970e-16
6,2025-11-11 21:00:00,Cipolletti,20.58,28,14.98,247,12,21,1,13.066,7.095,-7.071068e-01,7.071068e-01
7,2025-11-12 00:00:00,Cipolletti,16.71,42,12.69,248,7,0,2,11.139,6.298,0.000000e+00,1.000000e+00
8,2025-11-12 03:00:00,Cipolletti,15.04,47,11.56,251,0,3,2,10.340,6.018,7.071068e-01,7.071068e-01
9,2025-11-12 06:00:00,Cipolletti,13.90,50,10.02,244,0,6,2,8.573,4.967,1.000000e+00,6.123234e-17


## 📈 Visualización del comportamiento de la corrosión simulada

Para entender mejor el dataset `corrosion_df`, definimos una función `plot_corrosion`
que permite elegir entre tres visualizaciones:

1. **`"serie_tiempo"`**  
   Gráfico de línea de la `corrosion_rate` a lo largo del tiempo (`timestamp`).

2. **`"scatter_temp"`**  
   Dispersión de `temp` vs `corrosion_rate`, coloreado por `pollution_index`.  
   Sirve para ver cómo cambia la corrosión con la temperatura y el nivel de contaminación.

3. **`"box_hour"`**  
   Boxplot de `corrosion_rate` por `hour`, para ver en qué horas del día
   el ambiente resulta más agresivo en promedio.

La función recibe el DataFrame y un parámetro `kind` con el nombre del gráfico, y muestra el gráfico seleccionado.

In [83]:
 # Importo la libreria plotly

import plotly.express as px
import plotly.io as pio

pio.templates.default = 'plotly_dark'        # Pongo el tema oscuro como estilo por defecto

def plot_corrosion(df: pd.DataFrame, kind: str = "serie_tiempo"):
# Gráfico 1: línea de tiempo de corrosion_rate

    if kind == "1":
      fig = px.line(
            df,
            x="timestamp",                                                   # Eje X: el tiempo
            y="corrosion_rate",                                              # Eje Y: la tasa de corrosión simulada
            title="Evolucion temporal de la tasa de corrosion simulada",
            markers = True,                                                  # Pongo puntitos en cada observación
            hover_data = ["temp", "humidity", "pollution_index"],            # Las variables extras que se muestran al pasar el mouse
        )
      fig.update_layout(xaxis_title=None,
                        yaxis_title="Tasa de corrosión (unidades simuladas)")
      fig.show()

# Gráfico 2: scatter temperatura vs corrosión, coloreado por contaminación

    elif kind == "2":
      fig = px.scatter(
          df,
          x = "temp",                                                      # Eje X: temperatura
          y = "corrosion_rate",                                            # Eje Y: tasa de corrosión
          color = "pollution_index",                                       # Color del punto según índice de contaminación
          title = "Corrosion vs temperatura (color = indice de contaminacion)",
          labels = {"temp": "Temperatura (°C)",
                    "corrosion_rate": "Tasa de corrosión (unidades simuladas)",
                    "pollution_index": "Índice de contaminación",},
          hover_data=["humidity", "wind_speed", "clouds"],                 # Las variables extras que se muestran al pasar el mouse
        )
      fig.show()

# Gráfico 3: boxplot de corrosión por hora del día

    elif kind == "3":
      fig = px.box(
          df,
          x = "hour",                                                  # Eje X: hora del día (0, 3, 6, ...)
          y = "corrosion_rate",                                        # Eje Y: tasa de corrosión
          title = "Distribucion de corrosion por hora del dia",
          labels = {"hour": "Hora del día",
                    "corrosion_rate": "Tasa de corrosión (unidades simuladas)",},
          points='all',
      )
      fig.update_traces(boxmean="sd")

      fig.show()

# Opción 4: mostrar los tres gráficos uno detrás de otro

    elif kind == "4":
      plot_corrosion(df, kind="1")
      plot_corrosion(df, kind="2")
      plot_corrosion(df, kind="3")



corrosion_df = build_corrosion_dataset(forecast_raw)         #  Acá construyo el DataFrame final de corrosión a partir del pronóstico

# Bucle infinito hasta que el usuario elija salir

while True:
    choice = input(
        "\n¿Qué gráfico deseas ver?\n"
        " 1 - Evolución temporal de corrosión simulada\n"
        " 2 - Corrosión vs Temperatura (color = contaminación)\n"
        " 3 - Distribución de corrosión por hora del día\n"
        " 4 - Todos los gráficos\n"
        " q - Salir\n"
        " Elegí 1, 2, 3, 4 o q: "
    ).strip().lower()

    if choice in ("q", "salir"):                              # Si el usuario escribe 'q' o 'salir', corto el bucle

        print("Saliendo del visor de gráficos...")
        break

    try:
        plot_corrosion(corrosion_df, kind=choice)             # Intento dibujar el gráfico según la opción elegida
    except ValueError as e:
        print(f"Opción inválida: {e}")



¿Qué gráfico deseas ver?
 1 - Evolución temporal de corrosión simulada
 2 - Corrosión vs Temperatura (color = contaminación)
 3 - Distribución de corrosión por hora del día
 4 - Todos los gráficos
 q - Salir
 Elegí 1, 2, 3, 4 o q: 4



¿Qué gráfico deseas ver?
 1 - Evolución temporal de corrosión simulada
 2 - Corrosión vs Temperatura (color = contaminación)
 3 - Distribución de corrosión por hora del día
 4 - Todos los gráficos
 q - Salir
 Elegí 1, 2, 3, 4 o q: q
Saliendo del visor de gráficos...


## 🧱 Preparación de datos para el modelo

En esta sección tomo el DataFrame `corrosion_df` y lo separo en:

- **Variables de entrada (X)**:  
  Uso solo variables que vienen del clima y del tiempo:
  - `temp` (temperatura),
  - `humidity` (humedad),
  - `wind_speed` (velocidad del viento),
  - `clouds` (nubosidad),
  - `hour` (hora del día),
  - `dayofweek` (día de la semana),
  - `pollution_index` (índice sintético de contaminación).

- **Variable objetivo (y)**:  
  - `corrosion_rate`, la tasa de corrosión simulada que quiero predecir.

Como se trata de una serie temporal, no mezclo todo al azar:  
ordeno por tiempo y uso un **70% de los datos iniciales para entrenamiento** y el **30% final para test**, simulando “entreno con el pasado, pruebo con el futuro”.

In [74]:
from sklearn.model_selection import train_test_split

corrosion_df = build_corrosion_dataset(forecast_raw)  # Partimos del df ya armado

feature_cols = [                                      # Definimos las variables independientes para predecir corrosion_rate
    "temp",
    "humidity",
    "wind_speed",
    "clouds",
    "hour",
    "dayofweek",
    "pollution_index",
    "hour_sin",
    "hour_cos",
  ]

x = corrosion_df[feature_cols]                        # Asignamos valores a x
y = corrosion_df["corrosion_rate"]                    # Asignamos valores a y

n = len(corrosion_df)                                 # Cantidad total de filas que tiene el DataFrame de corrosión
split_inx = int(n * 0.7)                              # Índice de corte: uso el 70% de los datos para entrenar y el 30% final para test (si n = 40 → 40 * 0.7 = 28 → split_idx = 28)

x_train, x_test = x.iloc[:split_inx], x.iloc[split_inx:]    #  Separo las FEATURES (x) en train y test respetando el orden temporal
y_train, y_test = y.iloc[:split_inx], y.iloc[split_inx:]    # Separo la VARIABLE OBJETIVO (y) en train y test con el mismo corte
x_train.shape, x_test.shape                                 # Miro cuántas filas tiene cada parte (train y test)

((28, 9), (12, 9))

## 🤖 Modelado y evaluación: regresión lineal vs Random Forest

Con los datos ya separados en `X_train`, `X_test`, `y_train` y `y_test`:

1. **Entreno dos modelos supervisados de regresión:**
   - `LinearRegression`: modelo lineal simple, sirve como baseline.
   - `RandomForestRegressor`: modelo más flexible basado en muchos árboles de decisión.

2. **Evalúo ambos modelos en el conjunto de test** usando:
   - **MAE (Mean Absolute Error)**: error absoluto medio.  
     Indica cuánto se equivoca el modelo, en promedio, en unidades de `corrosion_rate`.
   - **R² (coeficiente de determinación)**:  
     mide qué proporción de la variación de la corrosión es explicada por el modelo (más cerca de 1 es mejor).










In [73]:
# Importo librerias
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# =========================
# 🔹 Modelo 1: Regresión lineal
# =========================


lin_reg = LinearRegression()                 # Creo el modelo de regresión lineal (la "cajita" vacía)
lin_reg.fit(x_train, y_train)                # Le enseño al modelo usando los datos de entrenamiento

y_pred_lin = lin_reg.predict(x_test)         # El modelo trata de adivinar y_test a partir de X_test


# Calculo el error medio absoluto (MAE) y el R² para la regresión lineal

mae_lin = mean_absolute_error(y_test, y_pred_lin)      # promedio de cuánto se equivoca, en unidades de corrosión
r2_lin = r2_score(y_test, y_pred_lin)                  # qué porcentaje de la variación explica el modelo


print("Regresión lineal")
print(" MAE:", round(mae_lin, 3))
print(" R² :", round(r2_lin, 3))


# =========================
# 🔹 Modelo 2: Random Forest
# =========================


rf = RandomForestRegressor(
    n_estimators=200,        # cantidad de árboles en el bosque
    random_state=42,         # semilla para que los resultados sean reproducibles
    n_jobs=-1)               # usa todos los núcleos disponibles para entrenar más rápido


rf.fit(x_train, y_train)      # Entreno el bosque con los mismos datos de entrenamiento

y_pred_rf = rf.predict(x_test)  # El Random Forest hace sus predicciones sobre el set de test

# Calculo las mismas métricas para el Random Forest
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\nRandom Forest")
print(" MAE:", round(mae_rf, 3))
print(" R² :", round(r2_rf, 3))

Regresión lineal
 MAE: 0.777
 R² : 0.872

Random Forest
 MAE: 1.345
 R² : 0.539


## 📊 Visualización del desempeño del modelo

Para el modelo final (regresión lineal) muestro dos gráficos que se complementan:

1. **Línea en el tiempo: corrosión real vs predicha**
   - Eje X: `timestamp` (tiempo).
   - Eje Y: `corrosion_rate`.
   - Una línea muestra la corrosión **real** y otra la **predicha** por el modelo.
   - Si las dos líneas van bastante pegadas, significa que el modelo está siguiendo bien el comportamiento de la corrosión a lo largo del tiempo.

2. **Dispersión: corrosión real vs predicha + línea ideal**
   - Eje X: valores **reales** de `corrosion_rate`.
   - Eje Y: valores **predichos** por el modelo.
   - Cada punto es un instante.  
   - Se agrega una línea diagonal `y = x` que representa el modelo perfecto:
     - si un punto cae sobre la diagonal → predicción exacta,
     - cuanto más se aleja de la diagonal → mayor error.
   - Esto permite ver de forma clara qué tan cerca están las predicciones del valor ideal.

Con estos dos gráficos puedo explicar visualmente cómo se comporta el modelo, además de las métricas numéricas (MAE y R²).








In [79]:


# Armamos un DataFrame con la parte de TEST
results_df = corrosion_df.iloc[split_inx:].copy()
results_df["real"] = y_test.values      # valores reales de corrosión (y_test)
results_df["pred_lin"] = y_pred_lin     # valores predichos por la regresión lineal


# Gráfico de líneas: real vs predicha en el tiempo ===
fig = px.line(
    results_df,
    x="timestamp",                                   # eje X: tiempo
    y=["real", "pred_lin"],                          # dos series: real y predicha
    title="Corrosión real vs predicha (Regresión lineal)",
    labels={"value": "Tasa de corrosión", "timestamp": "Tiempo"},
    template="plotly_dark",
)
fig.update_layout(legend_title_text="Serie")
fig.show()



# Buscamos el mínimo y máximo entre reales y predichos, y damos un margen de 1 unidad
lim_inf = min(y_test.min(), y_pred_lin.min()) - 1
lim_sup = max(y_test.max(), y_pred_lin.max()) + 1

# Gráfico de dispersión: valores reales (X) vs predichos (Y)
fig = px.scatter(x=y_test,                   # Eje X: valores reales
                 y=y_pred_lin,               # Eje Y: valores predichos
                 title = 'Comparación entre valores reales y predichos',
                 template = 'plotly_dark',
                 )
# Agrego una traza de puntos CON NOMBRE y estilo (para la leyenda)
fig.add_scatter(
    x=y_test,
    y=y_pred_lin,                             # mismos datos para superponer estilo/leyenda
    mode='markers',
    name='Predicciones',                      # 🔹 aparece en la leyenda
    marker=dict(color='royalblue', size=6)
)

# Línea ideal (y = x)

fig.add_scatter(x=[lim_inf, lim_sup],        # extremos de la diagonal
                y=[lim_inf, lim_sup],
                mode='lines',
                name='Línea ideal  (y = x)', # 🔹 leyenda
                line=dict(color='orange', dash='dash'))

# Etiquetas y leyenda
fig.update_layout(
    xaxis_title='Corrosión real',
    yaxis_title='Corrosión predicha',
    showlegend=True
)

fig.show()

### ✅ Conclusión del modelo

En estos gráficos se ve que:

- La **línea de la regresión lineal** sigue bastante bien a la corrosión real en el tiempo.
- En el gráfico de dispersión, la mayoría de los puntos queda **cerca de la diagonal** `y = x`, que representa el modelo perfecto (real = predicho).
- El error medio (MAE ≈ 0.78) es menor a 1 unidad en un rango aproximado de 0 a 9, y el modelo explica cerca del 87% de la variación (R² ≈ 0.87).

Probé también un modelo de **Random Forest**, pero en este dataset la corrosión simulada se construyó casi como una combinación lineal de las variables climáticas. Por eso, la **regresión lineal resulta más simple, interpretable y, en este caso, también más precisa**, así que enfoqué los gráficos en ese modelo.